In [7]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature = 0.1,
)

In [8]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content = "You are a geography expert. And you only reply in Italian."
        ),
        AIMessage(content = "Ciao, mi chiamo Paolo!"),
        HumanMessage(content="What is the distance between Mexico and Thailand? Also what is your name?"),
]

chat.predict_messages(messages)

AIMessage(content='La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Come posso aiutarti oggi?')

The issue in your code seems to be a typo in the string "How many palnets are there?". The correct spelling should be "planets". Here is the corrected code:

